In [1]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
import os

import truststore
truststore.inject_into_ssl()
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import seaborn as sns
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
os.environ["APP_CLIENT_ID"] = "zizhang-chen-research-app"
os.environ["APP_CLIENT_SECRET"] = "cQPcueFP7tDrimbf8NW2GAHcHeQa"
from llm_idam_token_generator.idam_token_generator import get_idam_token
from langchain_community.callbacks import get_openai_callback
from langchain_openai import OpenAI
import tiktoken

In [2]:
end_point_list = [
    'https://lmaas-beta.ai.gehealthcare.com',
    "https://openai-llm-frontdoor-hma7evbthrd4cugn.a01.azurefd.net"
]

model_list = ["gpt-35-turbo-16k", "gpt-4-32k-beta", "gpt-4o"]

# APIM_KEY = "8b96051ed6b84e4dad762fdc9f8c809e"

OPENAI_ENDPOINT = end_point_list[0]
OPENAI_DEPLOYMENT_MODEL = model_list[0]
OPENAI_AZURE_API_VERSION = "2023-12-01-preview"
# OPENAI_AZURE_API_VERSION = '2024-02-01'
OPENAI_TYPE = "azure"

In [3]:
# help(AzureChatOpenAI)

llm = AzureChatOpenAI(
    api_key="xxx",  # This is not playing any role, but required as per OpenAI sdk. So any random could be passed.
    azure_endpoint=OPENAI_ENDPOINT,
    deployment_name=OPENAI_DEPLOYMENT_MODEL,
    openai_api_version=OPENAI_AZURE_API_VERSION,
    n = 2,
    default_headers={
        'Authorization': f'Bearer {get_idam_token()}',
        'Content-Type': 'application/json'
    }
)

PID:12218 INFO llm_idam_token_generator.idam_token_generator - Client ID: zizhang-chen-research-app - Generating new token.
PID:12218 INFO llm_idam_token_generator.idam_token_generator - Client ID: zizhang-chen-research-app - All required environment variables are present.
/opt/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
PID:12218 INFO llm_idam_token_generator.idam_token_generator - Client ID: zizhang-chen-research-app - IDAM Access Token is generated
/opt/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-w

In [4]:
def generate_prompt_task(path):
    with open(path) as f:
        prompt = f.readlines()
    return prompt

def generate_cot():

    cot = ["1. **Review Patient Profile:** Analyze age, sex, and medical history for any factors that significantly increase ICU transfer risk.",
    "2. **Evaluate Current Symptoms:** Consider how severe the symptoms are and whether they suggest worsening of condition or stability.",
    "3. **Assess Vital Signs:** Identify any vital signs that are outside normal ranges and what they indicate about the patient's current health status.",
    "4. **Interpret Laboratory Results:** Look for abnormalities in lab results that could signal a need for intensive care, such as organ dysfunction or severe infection.",
    "5. **Consider Medical Events Since Admission:** Determine if the medical events reported since admission suggest a trend towards improvement or deterioration."]
    "6. **Make Prediction:** Based on the analysis, predict if ICU transfer is likely and justify the decision with specific medical reasoning."
    return '\n'.join(cot)

def combin_prompt_with_cot(general_prompt, cot, medical_event):
    template_by_task = '**Task:**\n' + general_prompt + '\n'
    
    template_cot = '**Chain of Thought Analysis:**\n'+ cot + '\n'

    template_events = '**Medical Events**\n' + medical_event + '\n'

    cot_template = template_by_task + '\n' + template_cot + '\n' + '{events}'
    return cot_template, template_events

def combin_prompt_no_cot(general_prompt, medical_event):
    template_by_task = '**Task:**\n' + general_prompt + '\n'
    
    template_events = '**Medical Events**\n' + medical_event + '\n'

    template_events_answer_style = "**Answer requirement:**\nPlease only answer with 'Yes' or 'No'"

    cot_template = template_by_task + '\n' + '{events}'
    return cot_template, template_events + '\n' + template_events_answer_style

def sample_indices(seed, array_A, sample_size, min_true_count):
    if array_A.size < sample_size:
        raise ValueError("The size of array A must be at least as large as the sample size.")
    if np.count_nonzero(array_A) < min_true_count:
        raise ValueError("Array A does not contain enough True values.")
    
    np.random.seed(seed)
    
    # Indices of True and False in array_A
    true_indices = np.where(array_A)[0]
    false_indices = np.where(~array_A)[0]
    
    if len(true_indices) < min_true_count:
        raise ValueError("Array A contains fewer than the minimum required True values.")

    selected_true_indices = np.random.choice(true_indices, min_true_count, replace=False)
    
    # Remaining indices needed to be picked
    remaining_indices_needed = sample_size - min_true_count

    remaining_pool = np.setdiff1d(np.arange(array_A.size), selected_true_indices)

    selected_other_indices = np.random.choice(remaining_pool, remaining_indices_needed, replace=False)

    final_indices = np.concatenate((selected_true_indices, selected_other_indices))
    
    return final_indices

def calculate_row_entropy(df):
    def entropy(row):
        counts = row.value_counts(normalize=True)
        return -np.sum(counts * np.log2(counts + np.finfo(float).eps)) 

    return df.apply(entropy, axis=1)

def filter_array_by_range(array, min_value, max_value):
    # Create a boolean mask where True values are those within the specified range
    mask = (array >= min_value) & (array <= max_value)
    
    # Use the mask to select elements from the array
    return mask

def num_tokens_from_string(string, encoding_model = 'gpt-3.5-turbo') -> int:
    encoding = tiktoken.encoding_for_model(encoding_model)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [5]:
# df = pd.read_csv('common_patients_description.csv')
# description_list = df['description']

# icu_patients = pd.read_csv('icu_labeled_patients.csv')
# icu_patients.rename(columns={'prediction_time': 'admission_date', 'value':'icu_label'}, inplace=True)

# los_patients = pd.read_csv('los_labeled_patients.csv')
# los_patients.rename(columns={'prediction_time': 'admission_date', 'value':'los_label'}, inplace=True)

# readmin_patients = pd.read_csv('readmin_labeled_patients.csv')
# readmin_patients.rename(columns={'prediction_time': 'discharge_date', 'value':'readmin_label'}, inplace=True)

# df2 = df.merge(icu_patients[['patient_id', 'admission_date', 'icu_label']], on=['patient_id', 'admission_date'], how='left')
# df2 = df2.merge(los_patients[['patient_id', 'admission_date', 'los_label']], on=['patient_id', 'admission_date'], how='left')
# df2 = df2.merge(readmin_patients[['patient_id', 'discharge_date', 'readmin_label']], on=['patient_id', 'discharge_date'], how='left')

# num_tokens =np.array([num_tokens_from_string(description_list[i], "gpt-3.5-turbo") for i in range(len(description_list))])
# valid_index = filter_array_by_range(num_tokens, 1000, 10000)
# df2 = df2[valid_index]
# df2 = df2.reset_index(drop=True)
# df2.to_csv('data/operation_patient_description.csv', index = False)
df2 = pd.read_csv('data/operation_patient_description.csv')
description_list = df2['description']

In [6]:
prompt_list = ['prompts/baseline/ICU_baseline.txt', 'prompts/baseline/LOS_baseline.txt', 'prompts/baseline/Readmin_baseline.txt']
label_name = ['icu_label', 'los_label', 'readmin_label']
i_list = pd.read_csv("results/gpt4/baseline_general_operation/ICU_baseline.csv")['orig_index'].values
cb_list = []
# for idx in [1]:
# for idx in range(1, 3):
for idx in range(3):
    out_csv_names = prompt_list[idx].split('/')[-1].replace('.txt', '.csv')
    general_prompt = generate_prompt_task(prompt_list[idx])[0]
    gt_array = df2[label_name[idx]]
    gt_list = gt_array[i_list]

    # pd.DataFrame(i_list).to_csv(os.path.join('indexs/', 'indexs_' + out_csv_names))

    with get_openai_callback() as cb:

        answer_list_all = []

        for i in tqdm(i_list):
            # for i in tqdm(range(1)):
            answer_list_5 = []
            for _ in range(5):
                c, e = combin_prompt_no_cot(general_prompt,
                                            description_list[i])
                prompt = ChatPromptTemplate.from_template(c)
                chain = prompt | llm
                result = chain.invoke({'events': e})
                answer_list_5.append(result.content)
            answer_list_all.append(answer_list_5)

        df_pred = pd.DataFrame(answer_list_all)
        df_pred.columns = ['pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5']
        df_pred['gt'] = gt_list.values
        df_pred['orig_index'] = i_list
        # df_pred.to_csv(
        #     os.path.join(
        #         'results/gpt4/baseline_general_operation_v2/',
        #         prompt_list[idx].split('/')[-1].replace('.txt', '.csv')))
    df_pred.to_csv(os.path.join('results/gpt35/baseline_general_operation_v2/', prompt_list[idx].split('/')[-1].replace('.txt', '.csv')))

  0%|          | 0/100 [00:00<?, ?it/s]

PID:12218 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://lmaas-beta.ai.gehealthcare.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:12218 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://lmaas-beta.ai.gehealthcare.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:12218 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://lmaas-beta.ai.gehealthcare.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:12218 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://lmaas-beta.ai.gehealthcare.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:12218 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://lmaas-beta.ai.gehealthc

In [7]:
print(e[-100:])

nous blood by venipuncture) occurred.

**Answer requirement:**
Please only answer with 'Yes' or 'No'


In [8]:
print(c)

**Task:**
You are an experienced doctor. Based on the provided patient age and medical events on the day of the admission, use your medical knowledge and reasoning to predict whether the patient will be readmitted in 30 days. Considering the critical factors that would influence such a decision. Please only answer with Yes or No. Yes if the patient will be readmitted in 30 days, No, if the patient will not be readmitted in 30 days.

{events}


In [9]:
# pd.DataFrame(answer_list_all).to_csv('results/ICU_baseline.csv')
# pd.DataFrame(answer_list_all).to_csv('results/LOS_baseline.csv')
# pd.DataFrame(answer_list_all).to_csv('results/readmin.csv')

# pd.DataFrame(answer_list_all).to_csv('results/ICU_baseline.csv')
# pd.DataFrame(answer_list_all).to_csv('results/LOS_baseline.csv')
# pd.DataFrame(answer_list_all).to_csv('results/gpt/35/readmin.csv')
# df_pred.to_csv(os.path.join('results/gpt35/baseline2/', prompt_list[idx].split('/')[-1].replace('.txt', '.csv')))

In [10]:
# df_pred = pd.DataFrame(answer_list_all)
# df_pred.columns = ['pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5']
# df_pred['gt'] = gt_list.values
# df_pred['orig_index'] = i_list
# df_pred.to_csv('results/readmin.csv')

In [11]:

# df_pred = pd.read_csv('results/LOS_baseline.csv')
# gt_array = df2['readmin_label']

# df_pred = pd.read_csv('results/ICU_baseline.csv')
# gt_array = df2['icu_label']

# df_pred = pd.read_csv('results/readmin.csv')
# gt_array = df2['readmin_label']
# gt_list = gt_array[i_list]

# df_pred_baseline = df_pred.loc[:, ['pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5']]
# df_pred_baseline.replace({'Yes': 1, 'No': 0}, inplace=True)
# uq_metric_baseline = calculate_row_entropy(df_pred_baseline).values
# uq_metric_baseline = np.round(uq_metric_baseline, 10) + 0.0000000001


# acc_list = []
# for c in df_pred_baseline.columns:
#     p = df_pred_baseline[c].values
#     acc = np.mean(df_pred_baseline[c] == df_pred['gt'].astype(int))
#     acc_list.append(acc)
# np.mean(acc_list), np.std(acc_list)

In [12]:
# def plot_auc_roc(y_true, scores):
    
#     fpr, tpr, _ = roc_curve(y_true, scores)
#     roc_auc = auc(fpr, tpr)

#     # Plotting
#     plt.figure()
#     plt.plot(fpr, tpr, color='darkorange',
#              lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
#     plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('Receiver Operating Characteristic')
#     plt.legend(loc="lower right")
#     plt.show()

In [13]:
# df_age = pd.DataFrame({'Age': age, 'ICU': pred_1})


# df_age['Age Group'] = pd.cut(df_age['Age'], bins=[0, 20, 40, 60, 80, 100], labels=['0-20', '21-40', '41-60', '61-80', '81-100'])


# plt.figure(figsize=(20, 6))
# sns.countplot(x='Age', hue='ICU', data=df_age)
# plt.title('Responses by Age Group')
# plt.xlabel('Age Group')
# plt.ylabel('Count of Responses')
# plt.legend(title='Response')
# plt.show()